<a href="https://colab.research.google.com/github/vshlemon/colabs/blob/main/notebooks/karpathy_nns/chapter_2_makemore_i.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/vshlemon/colabs.git

# Data exploration

In [ ]:
data_dir = '/content/colabs/notebooks/karpathy_nns/data'
words = open(f'{data_dir}/names.txt', 'r').read().splitlines()

In [ ]:
words[:5]

**Statistical information in words**

Karpathy mentions there is already a lot of statistical data to learn from in a word. For example in `isabella` we can look at which letter follows the preceding context and use that as data to train a model to learn statistical associations between:
  - i -> s
  - is -> a
  - isa -> b
  - ...
  - isabella -> [end]

So even just one name has a lot of training data.

In [ ]:
print(f"""
  Number of names: {len(words)},
  Smallest name size: {min([len(w) for w in words])} characters long,
  Largest name size: {max([len(w) for w in words])} characters long
""")

# Bigram Models

These are the simplest language models that can be learnt - they are simply models of the relationship between coupled characters. For e.g the bigrams in `Happy` are:
  - (H, a)
  - (a, p)
  - (p, p)
  - (p, y)

And we train a model over these bigrams to learn statistical associations of which character is likely to follow given the preceding one. Thus, we are conditioning on a history/context of a single character and outputting a single succeeding character.

In [ ]:
# printing the word and its succeeding word (by using the words original characters aligned
# with its characters offset by 1 - when the zip items are no longer aligned i.e. there is
# nothing in one of them, then the loop halts, that is why the last character has no follow
# up since there is nothing in the offset list to match with it)

print("Current character -> Succeeding character\n")
for w in words[:5]:
  print(f"Name: {w}")
  for ch_curr, ch_next in zip(w, w[1:]):
    print(f"""{ch_curr} -> {ch_next}""")
  print()

**Start and End Tokens**

Since we also want to learn patterns of the start of words so that when we have a blank space we have a statistical association of what might follow it to start a new word (and this also extends to learning associations for blank spaces preceded by characters eg. `hello _`), we add a start token `<S>`.

And to learn patterns for when to end words we add an end token `<E>` so we develop some statistical associations of which preceding sequence of characters might result in the termination of the word.

In [ ]:
print("Current character -> Succeeding character (inc. `<S>` & `<E>` tokens) \n")
for w in words[:5]:
  print(f"Name: {w}")
  chs = ['<S>'] + list(w) + ['<E>']
  for ch_curr, ch_next in zip(chs, chs[1:]):
    print(f"""{ch_curr} -> {ch_next}""")
  print()

**Collecting Statistics of Bigram Associations**

We can collect some statistics to understand better which letters most often precede or succeed others

In [ ]:
bigrams = {}
for w in words[:5]:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch_curr, ch_next in zip(chs, chs[1:]):
    bigram = (ch_curr, ch_next)
    bigrams[bigram] = bigrams.get(bigram, 0) + 1 # if the bigram has not been seen yet set it's default count to 0 & add 1, else get its current count and add 1
bigrams

For eg. we can see from the above that `<E>` i.e. the end of the name is most often preceded by `a` that is to say of the 5 names we collected statistics for, they all end in `a`. Now we will collect statistics for all of the names' bigrams.

In [ ]:
bigrams = {}
for w in words:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch_curr, ch_next in zip(chs, chs[1:]):
    bigram = (ch_curr, ch_next)
    bigrams[bigram] = bigrams.get(bigram, 0) + 1

In [ ]:
# we can also sort the bigrams my most frequent to least to get a bit more of a feel for them
# we do this by getting the items from the dictionary, and then using the value as the key to
# sort on but we use it's negative to sort as sorted by default sorts in ascending order & so
# the larger the negative number the earlier it will be in the order
sorted(bigrams.items(), key=lambda kv: -kv[1])[:5] # value is second element of key value (kv) pairs, we print just a few as there's a lot of possible bigrams

__Stretch Question__:
- How many possible bigrams are there with 26 alphabetic letters?
  - Since order matters here we are looking for permutations not combinations, the latter are distinct sets of numbers. Thus, since we are looking for groups of size 2 we do:
    - `26! * (1/(26-2)!)` which is effectively `26 * 25` i.e. **650** unique bigram pairings (this allows for things like `(a,b)` and `(b,a)` - if we wanted combinations where we only cared about the members not their order then we would further divide by `2!` to get _350_ combinations, since this eliminates any duplicative counts of sequences with the same members but in different orderings)

**Structuring as a Matrix & Visualising**
It is preferable to have our bigram counts in a matrix shape of rows and columns that we visualise so we can more easily identify the statistical associations. We will do this using PyTorch tensors as the library has good matrix methods and can be extended further on for our deep-learning requirements

In [ ]:
import torch

In [ ]:
MAP = torch.zeros((28, 28), dtype=torch.int32) # an zero initialised 28*28 size array with int32 precision (reason for 28*28 is 26 alphabets and `<S>` & `<E>` tokens)

In [ ]:
# we need a way to map from the characters to integers (so we can use them as indexes in our mapping matrix)
chars = sorted(list(set(''.join(words))))
stoi = {s:i for i,s in enumerate(chars)}
stoi['<S>'] = 26
stoi['<E>'] = 27
#stoi

In [ ]:
for w in words:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch_curr, ch_next in zip(chs, chs[1:]):
    ix1 = stoi[ch_curr]
    ix2 = stoi[ch_next]
    MAP[ix1, ix2] += 1 # adding a count of 1 every time we see a particular bigram pairing (remember MAP's values start at 0)
#MAP

In [ ]:
# creating a reverse mapping from integers to string so we can use the actual characters when needed
itos = {i:s for s,i in stoi.items()}
#itos

In [ ]:
# Visualising it
import matplotlib.pyplot as plt
%matplotlib inline

'''
Creates a bluescale image colormap of MAP (this gives shading to the cells
dependent on the value within the matrix at that cell - higher the value
darker the cell colour)

Then iterates through all possible bigram index combinations (nested loops
of 28 size each) and gets the characters at that location, and on the plot
places those characters as text at the index location, as well places the
count from the matrix collected above at the same index location
'''

plt.figure(figsize=(16,16))
plt.imshow(MAP, cmap='Blues')
for i in range(28):
    for j in range(28):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, MAP[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

You can see above that there is a redundant row at the very bottom since there is never another character immediately succeeding an end `<E>` character of a word, since it is always the last token of the word that indicates nothing else should come after.

You can also see there is a redundant column since the `<S>` token indicates a word about to begin so there is never anything before it, thus it can never appear at the second place in a bigram.

**Using a single special token**

We are going to utilise only a single special token to indicate both start and end of the word, and we will place it at the 0th (i.e. first) index of our alphabet - just out of a preference in visualising.

We are going to use `.` as our special token, this means this character will now signify the start and end of a word, so it will basically serve as word boundary.

This works since `<S>` is only used in one context of the bigram pairing, at the start, and `<E>` is only used in one context, at the end. And so effectively our new character `.` will serve as an `<S>` when learning associations for characters preceding the starting character of words and as an `<E>` when learning associations for characters succeeding the final character of words.

The statistical associations learnt will be along the lines of:
- Learn for a `.` to succeed words according to the statistics of it's pairing with the characters they end on - removing the redundancy of the `<S>` column
- Learn for `.` to preceed words according to the statistics of it's pairing with the characters they begin on - removing the redundancy of the `<E>` column

In [ ]:
MAP = torch.zeros((27, 27), dtype=torch.int32) # 27*27 is 26 alphabets and an `.` token

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)} # offset by 1, so `.` can take 0th index
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
for w in words:
  chs = ['.'] + list(w) + ['.'] # swapping out old start/end tokens for `.`
  for ch_curr, ch_next in zip(chs, chs[1:]):
    ix1 = stoi[ch_curr]
    ix2 = stoi[ch_next]
    MAP[ix1, ix2] += 1

In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(MAP, cmap='Blues')
for i in range(27): # now only 27 rows
    for j in range(27): # now only 27 cols
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, MAP[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

This is now much easier to interpret, the first row is the count of all starting letters of words (eg. `a` is the most popular starting letter of the names in our dataset) & the first column is the count of all ending letters of names (where `n` is the most common ending)